In [20]:
import cudf
import pandas as pd
import polars as pl
import modin.pandas as mp
import swifter
from pandarallel import pandarallel
from pathlib import Path
from numba import njit
from hurry.filesize import size
import numpy as np


pandarallel.initialize(progress_bar=False)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [127]:
%%time
data_root = Path('data')
stock_prices_path = data_root / 'stock_prices.csv'

stock_prices_pd = pd.read_csv(stock_prices_path, parse_dates=['Date']).dropna(subset=['Close'])
stock_prices_modin = mp.read_csv(stock_prices_path, parse_dates=['Date']).dropna(subset=['Close'])
stock_prices_cudf = cudf.read_csv(stock_prices_path, parse_dates=['Date']).dropna(subset=['Close'])
stock_prices_polars = pl.read_csv(stock_prices_path, parse_dates=True).drop_nulls(subset=['Close'])

CPU times: user 5.05 s, sys: 977 ms, total: 6.03 s
Wall time: 4.17 s


In [108]:
pd_concatted = pd.concat([stock_prices_pd, stock_prices_pd])
pd_concatted = pd.concat([pd_concatted, pd_concatted])
pd_concatted

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098


In [15]:
pd_concatted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9299692 entries, 0 to 2332530
Data columns (total 12 columns):
 #   Column            Dtype         
---  ------            -----         
 0   RowId             object        
 1   Date              datetime64[ns]
 2   SecuritiesCode    int64         
 3   Open              float64       
 4   High              float64       
 5   Low               float64       
 6   Close             float64       
 7   Volume            int64         
 8   AdjustmentFactor  float64       
 9   ExpectedDividend  float64       
 10  SupervisionFlag   bool          
 11  Target            float64       
dtypes: bool(1), datetime64[ns](1), float64(7), int64(2), object(1)
memory usage: 860.3+ MB


In [128]:
modin_concatted = mp.concat([stock_prices_modin, stock_prices_modin])
modin_concatted = mp.concat([modin_concatted, modin_concatted])
modin_concatted

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098


In [16]:
modin_concatted.info()

<class 'modin.pandas.dataframe.DataFrame'>
Int64Index: 9299692 entries, 0 to 2332530
Data columns (total 12 columns):
 #   Column            Non-Null Count    Dtype         
---  ----------------  ----------------  -----         
 0   RowId             9299692 non-null  object
 1   Date              9299692 non-null  datetime64[ns]
 2   SecuritiesCode    9299692 non-null  int64
 3   Open              9299692 non-null  float64
 4   High              9299692 non-null  float64
 5   Low               9299692 non-null  float64
 6   Close             9299692 non-null  float64
 7   Volume            9299692 non-null  int64
 8   AdjustmentFactor  9299692 non-null  float64
 9   ExpectedDividend  75364 non-null    float64
 10  SupervisionFlag   9299692 non-null  bool 
 11  Target            9299692 non-null  float64
dtypes: float64(7), int64(2), object(1), datetime64[ns](1), bool(1)
memory usage: 860.3 MB


In [57]:
cudf_concatted = cudf.concat([stock_prices_cudf, stock_prices_cudf])
cudf_concatted = cudf.concat([cudf_concatted, cudf_concatted])
cudf_concatted

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,<NA>,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,<NA>,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,<NA>,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,<NA>,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,<NA>,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,<NA>,False,0.034816
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,<NA>,False,0.025478
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,<NA>,False,-0.004302
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,<NA>,False,0.009098


In [17]:
cudf_concatted.info()

<class 'cudf.core.dataframe.DataFrame'>
Int64Index: 9299692 entries, 0 to 2332530
Data columns (total 12 columns):
 #   Column            Dtype
---  ------            -----
 0   RowId             object
 1   Date              datetime64[ns]
 2   SecuritiesCode    int64
 3   Open              float64
 4   High              float64
 5   Low               float64
 6   Close             float64
 7   Volume            int64
 8   AdjustmentFactor  float64
 9   ExpectedDividend  float64
 10  SupervisionFlag   bool
 11  Target            float64
dtypes: bool(1), datetime64[ns](1), float64(7), int64(2), object(1)
memory usage: 941.2+ MB


In [58]:
polars_concatted = pl.concat([stock_prices_polars, stock_prices_polars])
polars_concatted = pl.concat([polars_concatted, polars_concatted])
polars_concatted

RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
str,date,i64,f64,f64,f64,f64,i64,f64,str,bool,f64
"""20170104_1301""",2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,null,false,0.0007
"""20170104_1332""",2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,null,false,0.012324
"""20170104_1333""",2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,null,false,0.006154
"""20170104_1376""",2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,null,false,0.011053
"""20170104_1377""",2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,null,false,0.003026
"""20170104_1379""",2017-01-04,1379,2105.0,2147.0,2101.0,2143.0,77300,1.0,null,false,0.005169
"""20170104_1381""",2017-01-04,1381,1950.0,1960.0,1949.0,1960.0,1300,1.0,null,false,-0.009326
"""20170104_1407""",2017-01-04,1407,857.0,877.0,851.0,866.0,147000,1.0,null,false,-0.003437
"""20170104_1414""",2017-01-04,1414,4940.0,5060.0,4935.0,5050.0,119600,1.0,null,false,0.0


In [19]:
size(polars_concatted.estimated_size())

'862M'

In [7]:
cols = dict(zip(polars_concatted.columns, range(len(polars_concatted.columns))))
cols

{'RowId': 0,
 'Date': 1,
 'SecuritiesCode': 2,
 'Open': 3,
 'High': 4,
 'Low': 5,
 'Close': 6,
 'Volume': 7,
 'AdjustmentFactor': 8,
 'ExpectedDividend': 9,
 'SupervisionFlag': 10,
 'Target': 11}

In [21]:
def get_spread_ratio(
    high: np.ndarray,
    low: np.ndarray,
    close: np.ndarray,
) -> np.ndarray:
    return (high - low) / close

In [22]:
get_spread_ratio_numba = njit(get_spread_ratio)

In [8]:
%%timeit
pd_concatted.apply(lambda x: (x['High'] - x['Low']) / x['Close'], axis=1)

1min 47s ± 312 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
pd_concatted.swifter.apply(lambda x: (x['High'] - x['Low']) / x['Close'], axis=1)

In [23]:
%%timeit
pd_concatted.parallel_apply(lambda x: (x['High'] - x['Low']) / x['Close'], axis=1)

12.3 s ± 80.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%%timeit
modin_concatted.apply(lambda x: (x['High'] - x['Low']) / x['Close'], axis=1)

9.58 s ± 301 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%%timeit
cudf_concatted.apply(lambda x: (x['High'] - x['Low']) / x['Close'], axis=1)

5.06 ms ± 391 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit
polars_concatted.select((pl.col('High') - pl.col('Low')) / pl.col('Close'))

44.2 ms ± 92.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%%timeit
polars_concatted.apply(lambda x: (x[cols['High']] - x[cols['Low']]) / x[cols['Close']])

21.3 s ± 141 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit
(pd_concatted['High'] - pd_concatted['Low']).div(pd_concatted['Close'])

20.2 ms ± 218 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
%%timeit
get_spread_ratio(pd_concatted['High'], pd_concatted['Low'], pd_concatted['Close'])

20.3 ms ± 187 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%%timeit
get_spread_ratio_numba(pd_concatted['High'].to_numpy(), pd_concatted['Low'].to_numpy(), pd_concatted['Close'].to_numpy())

27.1 ms ± 145 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
%%timeit
(modin_concatted['High'] - modin_concatted['Low']).div(modin_concatted['Close'])

36.5 s ± 430 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%%timeit
(cudf_concatted['High'] - cudf_concatted['Low']).div(cudf_concatted['Close'])

5.62 ms ± 2.46 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [36]:
%%timeit
(polars_concatted['High'] - polars_concatted['Low']) / polars_concatted['Close']

46.6 ms ± 474 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
%%timeit
pd_concatted['High'].apply(lambda x: x ** 2)

1.51 s ± 8.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit
pd_concatted['High'].swifter.apply(lambda x: x ** 2)

8.86 ms ± 151 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%%timeit
pd_concatted['High'].parallel_apply(lambda x: x ** 2)

2.15 s ± 24.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%%timeit
modin_concatted['High'].apply(lambda x: x ** 2)

3.94 s ± 28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
%%timeit
cudf_concatted['High'].apply(lambda x: x ** 2)

3.87 ms ± 55.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
%%timeit
polars_concatted['High'].apply(lambda x: x ** 2)

1.57 s ± 9.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%%timeit
pd_concatted['High'] ** 2

7.62 ms ± 121 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
%%timeit
modin_concatted['High'] ** 2

3 s ± 13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
%%timeit
cudf_concatted['High'] ** 2

4.84 ms ± 104 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [46]:
%%timeit
polars_concatted['High'] ** 2

22.5 ms ± 234 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [47]:
%%timeit
pd_concatted.groupby(['Date', 'SecuritiesCode']).mean()

961 ms ± 6.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
%%timeit
modin_concatted.groupby(['Date', 'SecuritiesCode']).mean()

2022-07-12 01:53:13,581 - distributed.worker_memory - WARNING - Worker is at 87% memory usage. Pausing worker.  Process memory: 3.44 GiB -- Worker memory limit: 3.93 GiB
2022-07-12 01:53:13,625 - distributed.worker_memory - WARNING - Worker is at 76% memory usage. Resuming worker. Process memory: 3.01 GiB -- Worker memory limit: 3.93 GiB
2022-07-12 01:53:13,726 - distributed.worker_memory - WARNING - Worker is at 86% memory usage. Pausing worker.  Process memory: 3.39 GiB -- Worker memory limit: 3.93 GiB
2022-07-12 01:53:14,666 - distributed.worker_memory - WARNING - Worker is at 54% memory usage. Resuming worker. Process memory: 2.12 GiB -- Worker memory limit: 3.93 GiB
2022-07-12 01:53:20,190 - distributed.worker_memory - WARNING - Worker is at 90% memory usage. Pausing worker.  Process memory: 3.54 GiB -- Worker memory limit: 3.93 GiB
2022-07-12 01:53:20,252 - distributed.worker_memory - WARNING - Worker is at 79% memory usage. Resuming worker. Process memory: 3.13 GiB -- Worker mem

CancelledError: lambda-320b353f341d6f7dd900099bdf61142a

In [49]:
%%timeit
cudf_concatted.groupby(['Date', 'SecuritiesCode']).mean()

62.7 ms ± 588 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [50]:
%%timeit
polars_concatted.groupby(['Date', 'SecuritiesCode']).mean()

488 ms ± 9.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [110]:
trades_path = data_root / 'trades.csv'

trades_pd = pd.read_csv(trades_path, parse_dates=['Date']).dropna(subset=['TotalSales'])
trades_pd = pd.concat([trades_pd, trades_pd])
trades_pd = pd.concat([trades_pd, trades_pd])

trades_modin = mp.read_csv(trades_path, parse_dates=['Date']).dropna(subset=['TotalSales'])
trades_modin = mp.concat([trades_modin, trades_modin])
trades_modin = mp.concat([trades_modin, trades_modin])

trades_cudf = cudf.read_csv(trades_path, parse_dates=['Date']).dropna(subset=['TotalSales'])
trades_cudf = cudf.concat([trades_cudf, trades_cudf])
trades_cudf = cudf.concat([trades_cudf, trades_cudf])

trades_polars = pl.read_csv(trades_path, parse_dates=True).drop_nulls(subset=['TotalSales'])
trades_polars = pl.concat([trades_polars, trades_polars])
trades_polars = pl.concat([trades_polars, trades_polars])

In [97]:
def join_dataframes(df1, df2):
    return (df1
            .groupby(['Date', 'SecuritiesCode'])
            .mean()
            .merge(df2.groupby(['Date']).mean(), how='left', on='Date')
            .fillna(method='ffill')
            )

def join_dataframes_polars(df1, df2):
    return (df1
            .groupby(['Date', 'SecuritiesCode'])
            .mean()
            .join(df2.groupby(['Date']).mean(), how='left', on='Date')
            .fill_null("forward")
            )

In [102]:
%%timeit
join_dataframes(pd_concatted.iloc[:10_000], trades_pd)

7.02 ms ± 158 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [103]:
%%timeit
join_dataframes(pd_concatted.iloc[:100_000], trades_pd)

50 ms ± 469 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [104]:
%%timeit
join_dataframes(pd_concatted.iloc[:1_000_000], trades_pd)

495 ms ± 4.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [98]:
%%timeit
join_dataframes(pd_concatted, trades_pd)

1.62 s ± 15.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [111]:
%%timeit
join_dataframes(modin_concatted.iloc[:10_000], trades_modin)

4.17 s ± 83.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [112]:
%%timeit
join_dataframes(modin_concatted.iloc[:100_000], trades_modin)

4.37 s ± 118 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [113]:
%%timeit
join_dataframes(modin_concatted.iloc[:1_000_000], trades_modin)

5.91 s ± 254 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [114]:
%%timeit
join_dataframes(modin_concatted, trades_modin)

2022-07-12 02:30:03,333 - distributed.worker_memory - WARNING - Worker is at 89% memory usage. Pausing worker.  Process memory: 3.51 GiB -- Worker memory limit: 3.93 GiB
2022-07-12 02:30:03,445 - distributed.worker_memory - WARNING - Worker is at 76% memory usage. Resuming worker. Process memory: 3.01 GiB -- Worker memory limit: 3.93 GiB
2022-07-12 02:30:03,674 - distributed.worker_memory - WARNING - Worker is at 83% memory usage. Pausing worker.  Process memory: 3.30 GiB -- Worker memory limit: 3.93 GiB
2022-07-12 02:30:04,408 - distributed.worker_memory - WARNING - Worker is at 50% memory usage. Resuming worker. Process memory: 1.97 GiB -- Worker memory limit: 3.93 GiB
2022-07-12 02:30:16,427 - distributed.worker_memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 3.15 GiB -- Worker memory limit: 3.93 GiB
2022-07-12 02:30:16,536 - distributed.worker_memory - WARNING - Worker is at 77% memory usage. Resuming worker. Process memory: 3.03 GiB -- Worker mem

CancelledError: lambda-d367b2fbf633f0c8ed06b76cfa1a2b20

In [120]:
%%timeit
join_dataframes(cudf_concatted.iloc[:10_000], trades_cudf)

31.6 ms ± 314 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [121]:
%%timeit
join_dataframes(cudf_concatted.iloc[:100_000], trades_cudf)

41.6 ms ± 502 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [122]:
%%timeit
join_dataframes(cudf_concatted.iloc[:1_000_000], trades_cudf)

93.3 ms ± 2.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [118]:
%%timeit
join_dataframes(cudf_concatted, trades_cudf)

186 ms ± 3.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [123]:
%%timeit
join_dataframes_polars(polars_concatted.limit(10_000), trades_polars)

9.17 ms ± 122 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [124]:
%%timeit
join_dataframes_polars(polars_concatted.limit(100_000), trades_polars)

23.9 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [125]:
%%timeit
join_dataframes_polars(polars_concatted.limit(1_000_000), trades_polars)

266 ms ± 5.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [92]:
%%timeit
join_dataframes_polars(polars_concatted, trades_polars)

971 ms ± 21.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [132]:
%%timeit
pd_concatted.to_csv('data/test_pd.csv')

59.1 s ± 473 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [129]:
%%timeit
modin_concatted.to_csv('data/test_modin.csv')

1min 5s ± 326 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [131]:
%%timeit
cudf_concatted.to_csv('data/test_cudf.csv')

1.08 s ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [133]:
%%timeit
polars_concatted.write_csv('data/test_polars.csv')

749 ms ± 34.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [134]:
%%timeit
pd.read_csv('data/test_pd.csv')

7.4 s ± 41.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [135]:
%%timeit
mp.read_csv('data/test_modin.csv')

3.8 s ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [136]:
%%timeit
cudf.read_csv('data/test_cudf.csv')

325 ms ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [138]:
%%timeit
pl.read_csv('data/test_polars.csv')

427 ms ± 26.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
